<font size = "5">

Volatility Trend Trading Strategy

In [20]:
import pandas as pd
import numpy as np

<font size = "4">

Step 0: Collect our data from our priceData function. For this walkthrough we will use ``NFLX`` Netflix as an example.

In [21]:
from quantlab.data.yf import priceData
from quantlab.data.tickers import tickers

data = priceData(tickers, 150)
s = pd.Series(data["NFLX"])

[*********************100%***********************]  239 of 239 completed

23 Failed downloads:
['STLA', 'CNI', 'ORCL', 'BXP', 'GEHC']: Timeout('Failed to perform, curl: (28) Connection timed out after 10001 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['EQR', 'HASI', 'PG', 'JPM', 'MDB', 'FRT', 'UNP', 'NKE', 'INTC', 'EOG', 'EQIX', 'RHHBY', 'TROW', 'GS', 'VZ', 'MRK']: Timeout('Failed to perform, curl: (28) Connection timed out after 10002 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['ASML', 'AAPL']: Timeout('Failed to perform, curl: (28) Connection timed out after 10003 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


<font size = "4">

Step 1: Get a logarithmic series ``y`` of the prices. Then, calculate the short term and long term volatility, stored as ``vol_short`` and ``vol_long`` by taking the standard deviations of the daily returns of the last 60 days and 10 days respectively. Calculate the ``vol_ratio`` of the short term volatility divided by the long term volatility.

In [22]:
y = np.log(s)
vol_short = y[-10:].std()
vol_long = y[-60:].std()
vol_ratio = vol_short / vol_long

In [23]:
print(f"y: {np.array(y[:5])} ...")
print(f"vol_short: {vol_short}")
print(f"vol_long: {vol_long}")
print(f"vol_ratio: {vol_ratio}")

y: [7.10576971 7.10491614 7.12259246 7.13131476 7.12405142] ...
vol_short: 0.051800799278630114
vol_long: 0.032618540149011714
vol_ratio: 1.5880784070037417


<font size = "4">

Step 2: Use the ``current_price`` and the ``past_price`` from 10 days ago to calculate the direction of the stock, as ``price_movement`` (1 for increasing, 0 for decreasing).

In [24]:
current_price = s.iloc[-1]
past_price = s.iloc[-10]
price_movement = np.sign(current_price - past_price)

In [25]:
print(f"current_price: {current_price}")
print(f"past_price: {past_price}")
print(f"price_movement: {price_movement}")

current_price: 1102.5
past_price: 1203.2900390625
price_movement: -1.0


<font size = "4">
Step 3: If the short term volitility is greater than the long term volitility by at least a factor of 1.25x, the stock is believed to be entering a price shock, and should be expected to continue moving in its direction for a while longer. In this case, trade the stock depending on the direction it is moving. If not, the stock is in a calm state and isn't expected to move any time soon, so we don't do anything. The amount we buy should be standardized by price, so we divide 100 by the price of a share in order to buy/sell $100 of each stock.

In [26]:
if vol_ratio > 1.25:
    result = price_movement * (100 / current_price)
    print("Since the short term volitility is sufficiently higher than the long term volitility, make a move now.")
else:
    print("Since the short term volitility is not sufficiently higher than the long term volitility, do nothing.")

Since the short term volitility is sufficiently higher than the long term volitility, make a move now.


<font size = "4">
Now, Wrap all of this as a function, looping across all columns. Start by initializing an empty data frame ``results``, which will be added to if our result consists of us buying or selling a stock. Return the data frame of trading decisions of all stocks.

In [27]:
def momentum(data):
    results = {}
    for col in data.columns:
        s = pd.Series(data[col])
        current_price = s.iloc[-1]
        past_price = s.iloc[-8]
        change = np.log(current_price / past_price)
        daily_returns = np.log(s/s.shift(1))
        vol = daily_returns.iloc[-8:].std(ddof = 1)
        threshold = 0.75 * vol * np.sqrt(8)

        if change > threshold:
            results[col] = 100 / current_price
        elif change < -threshold:
            results[col] = -100 / current_price
        else:
            continue
    return pd.Series(results, name = "shares")

In [28]:
results = momentum(data)
print(results)

A       0.685636
ADBE    0.277847
AEP    -0.868734
AJG    -0.373832
ALL    -0.523972
          ...   
WFC     1.150748
WPC    -1.512630
XEL    -1.252819
XOM     0.869338
ZM      1.177995
Name: shares, Length: 101, dtype: float64
